In [1]:
import os
import re
import shutil
import string
import time
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import Sequential
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
import tensorflow_hub as hub

In [3]:
!pip install -q --upgrade keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 96.9 MB/s eta 0:00:00


In [4]:
import keras_nlp

In [5]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
#dataset_dir ="C:\\TEMP\\IMDB\\aclImdb"

84125825/84125825 [==============================] - 8s 0us/step


In [6]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)
print(train_dir)

./aclImdb/train


In [7]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [8]:
batch_size = 32
seed = 42

raw_train_ds0 = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=None,
    validation_split=0.2,
    subset='training',
    seed=seed). batch(batch_size,drop_remainder=True)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [9]:
raw_val_ds0 = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=None,
    validation_split=0.2,
    subset='validation',
    seed=seed). batch(batch_size,drop_remainder=True)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
print(f"Number of batches in raw_train_ds: {raw_train_ds0.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds0.cardinality()}")

Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 156


In [ ]:
for text_batch, label_batch in raw_train_ds0.take(1):
    print(len(text_batch))
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

32
b"Having seen most of Ringo Lam's films, I can say that this is his best film to date, and the most unusual. It's a ancient china period piece cranked full of kick-ass martial arts, where the location of an underground lair full of traps and dungeons plays as big a part as any of the characters. The action is fantastic, the story is tense and entertaining, and the set design is truely memorable. Sadly, Burning Paradise has not been made available on DVD and vhs is next-to-impossible to get your mitts on, even if you near the second biggest china-town in North America (like I do). If you can find it, don't pass it up."
1
b'Caution: May contain spoilers...<br /><br />I\'ve seen this movie 3 times & I\'ve liked it every time. Upon seeing it again, I\'m always reminded of how good it is. An HBO TV movie- very well done like most of their movies are- this would\'ve gotten Oscars for it\'s performances had it been released for general distribution instead of made for TV.<br /><br />As I\'

In [10]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),'')

In [11]:
raw_train_ds = raw_train_ds0.map(lambda x, y: (custom_standardization(x), y))
raw_val_ds = raw_val_ds0.map(lambda x, y: (custom_standardization(x), y))

In [ ]:
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")

Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157


In [12]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=None,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [13]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [14]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [15]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [16]:
embeddingDimensions = 16
vocablaryLength=len(vectorize_layer.get_vocabulary())

In [17]:
def compileAndFit(train_ds,val_ds, getModel,learning_rate):

    results=[]
    for attempt in range(3):
        model=getModel()

        s = time.time()

        model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

        #model.summary()

        history = model.fit(    train_ds,    validation_data=val_ds,    epochs=1000, verbose=1,
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5,mode='max',restore_best_weights=True)] )

        secs=time.time()-s
        lastEpoch=len(history.history['val_binary_accuracy'])

        history_dict = history.history

        loss, accuracy = model.evaluate(val_ds)
        tloss, taccuracy = model.evaluate(train_ds)
        results.append((secs,taccuracy,accuracy,lastEpoch))

    print("\t".join(['Time,s','Training accuracy','Validation accuracy','Epochs']))
    for row in results:
        print("\t".join([str(round(row[0])),str(round(row[1], 3)),str(round(row[2], 3)),str( row[3] )  ]))


# [Basic text classification](https://www.tensorflow.org/tutorials/keras/text_classification)

In [ ]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
      layers.Embedding(vocablaryLength, 16),
      layers.Dropout(0.2),
      layers.GlobalAveragePooling1D(),
      layers.Dropout(0.2),
      layers.Dense(1)]),0.001)

Epoch 1/1000
625/625 [==============================] - 61s 95ms/step - loss: 0.6661 - binary_accuracy: 0.6922 - val_loss: 0.6175 - val_binary_accuracy: 0.7722
Epoch 2/1000
625/625 [==============================] - 19s 30ms/step - loss: 0.5499 - binary_accuracy: 0.8005 - val_loss: 0.5002 - val_binary_accuracy: 0.8207
Epoch 3/1000
625/625 [==============================] - 11s 18ms/step - loss: 0.4447 - binary_accuracy: 0.8457 - val_loss: 0.4212 - val_binary_accuracy: 0.8470
Epoch 4/1000
625/625 [==============================] - 8s 13ms/step - loss: 0.3785 - binary_accuracy: 0.8649 - val_loss: 0.3743 - val_binary_accuracy: 0.8580
Epoch 5/1000
625/625 [==============================] - 8s 13ms/step - loss: 0.3361 - binary_accuracy: 0.8792 - val_loss: 0.3464 - val_binary_accuracy: 0.8646
Epoch 6/1000
625/625 [==============================] - 8s 13ms/step - loss: 0.3051 - binary_accuracy: 0.8888 - val_loss: 0.3263 - val_binary_accuracy: 0.8712
Epoch 7/1000
625/625 [=====================

# [Visualizing Data using the Embedding Projector in TensorBoard](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin)
# [Word embeddings](https://www.tensorflow.org/text/guide/word_embeddings)
# [Graph regularization for sentiment classification using synthesized graphs](https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb)

In [ ]:
compileAndFit(train_ds,val_ds,lambda:Sequential(    [
        layers.Embedding(vocablaryLength, 16),
        layers.GlobalAveragePooling1D(),
        layers.Dense(16, activation="relu"),
        layers.Dense(1),
    ]),0.001)

Epoch 1/1000
625/625 [==============================] - 48s 76ms/step - loss: 0.5808 - binary_accuracy: 0.7377 - val_loss: 0.4078 - val_binary_accuracy: 0.8440
Epoch 2/1000
625/625 [==============================] - 17s 26ms/step - loss: 0.3221 - binary_accuracy: 0.8788 - val_loss: 0.3087 - val_binary_accuracy: 0.8736
Epoch 3/1000
625/625 [==============================] - 11s 17ms/step - loss: 0.2473 - binary_accuracy: 0.9053 - val_loss: 0.2945 - val_binary_accuracy: 0.8774
Epoch 4/1000
625/625 [==============================] - 9s 14ms/step - loss: 0.2071 - binary_accuracy: 0.9233 - val_loss: 0.2957 - val_binary_accuracy: 0.8774
Epoch 5/1000
625/625 [==============================] - 6s 9ms/step - loss: 0.1770 - binary_accuracy: 0.9366 - val_loss: 0.2940 - val_binary_accuracy: 0.8794
Epoch 6/1000
625/625 [==============================] - 8s 12ms/step - loss: 0.1531 - binary_accuracy: 0.9467 - val_loss: 0.3040 - val_binary_accuracy: 0.8776
Epoch 7/1000
625/625 [======================

# [Graph regularization for sentiment classification using synthesized graphs](https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb/)

In [ ]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
   layers.Embedding(vocablaryLength,16),
   layers.Bidirectional( layers.LSTM(64)),
   layers.Dense(64, activation='relu'),
   layers.Dense(1)
]),0.001)

Epoch 1/1000
625/625 [==============================] - 66s 95ms/step - loss: 0.4752 - binary_accuracy: 0.7696 - val_loss: 0.3897 - val_binary_accuracy: 0.8339
Epoch 2/1000
625/625 [==============================] - 27s 43ms/step - loss: 0.2958 - binary_accuracy: 0.8870 - val_loss: 0.3836 - val_binary_accuracy: 0.8433
Epoch 3/1000
625/625 [==============================] - 22s 36ms/step - loss: 0.2259 - binary_accuracy: 0.9176 - val_loss: 0.3316 - val_binary_accuracy: 0.8678
Epoch 4/1000
625/625 [==============================] - 20s 31ms/step - loss: 0.1652 - binary_accuracy: 0.9433 - val_loss: 0.3825 - val_binary_accuracy: 0.8510
Epoch 5/1000
625/625 [==============================] - 19s 31ms/step - loss: 0.1388 - binary_accuracy: 0.9530 - val_loss: 0.3877 - val_binary_accuracy: 0.8584
Epoch 6/1000
625/625 [==============================] - 21s 33ms/step - loss: 0.1227 - binary_accuracy: 0.9589 - val_loss: 0.4585 - val_binary_accuracy: 0.8480
Epoch 7/1000
625/625 [==================

# My own RNN1

In [15]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
     layers.Embedding(vocablaryLength, 64, mask_zero=True),
     layers.Bidirectional( layers.LSTM(64)),
     layers.Dense(1)
]),0.0001)

Epoch 1/1000
625/625 [==============================] - 98s 132ms/step - loss: 0.5857 - binary_accuracy: 0.6690 - val_loss: 0.4183 - val_binary_accuracy: 0.8299
Epoch 2/1000
625/625 [==============================] - 35s 56ms/step - loss: 0.3407 - binary_accuracy: 0.8643 - val_loss: 0.3509 - val_binary_accuracy: 0.8656
Epoch 3/1000
625/625 [==============================] - 26s 42ms/step - loss: 0.2573 - binary_accuracy: 0.9040 - val_loss: 0.3301 - val_binary_accuracy: 0.8688
Epoch 4/1000
625/625 [==============================] - 25s 40ms/step - loss: 0.2121 - binary_accuracy: 0.9220 - val_loss: 0.3500 - val_binary_accuracy: 0.8574
Epoch 5/1000
625/625 [==============================] - 23s 36ms/step - loss: 0.1798 - binary_accuracy: 0.9340 - val_loss: 0.3673 - val_binary_accuracy: 0.8620
Epoch 6/1000
625/625 [==============================] - 22s 35ms/step - loss: 0.1516 - binary_accuracy: 0.9459 - val_loss: 0.3239 - val_binary_accuracy: 0.8692
Epoch 7/1000
625/625 [=================

KeyboardInterrupt: ignored

# My own RNN2

In [17]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
     layers.Embedding(vocablaryLength, 64, mask_zero=True),
     layers.LSTM(64),
     layers.Dense(1)
]),0.0001)

Epoch 1/1000
625/625 [==============================] - 64s 93ms/step - loss: 0.5909 - binary_accuracy: 0.6811 - val_loss: 0.4261 - val_binary_accuracy: 0.8289
Epoch 2/1000
625/625 [==============================] - 22s 36ms/step - loss: 0.3467 - binary_accuracy: 0.8702 - val_loss: 0.3496 - val_binary_accuracy: 0.8638
Epoch 3/1000
625/625 [==============================] - 17s 27ms/step - loss: 0.2553 - binary_accuracy: 0.9067 - val_loss: 0.3090 - val_binary_accuracy: 0.8748
Epoch 4/1000
625/625 [==============================] - 16s 26ms/step - loss: 0.2074 - binary_accuracy: 0.9283 - val_loss: 0.3143 - val_binary_accuracy: 0.8750
Epoch 5/1000
625/625 [==============================] - 15s 24ms/step - loss: 0.1737 - binary_accuracy: 0.9424 - val_loss: 0.3499 - val_binary_accuracy: 0.8696
Epoch 6/1000
625/625 [==============================] - 17s 27ms/step - loss: 0.1452 - binary_accuracy: 0.9542 - val_loss: 0.3313 - val_binary_accuracy: 0.8716
Epoch 7/1000
625/625 [==================

KeyboardInterrupt: ignored

# My own RNN3

In [18]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
     layers.Embedding(vocablaryLength,8 ),
     layers.LSTM(8),
     layers.Dense(1)
]),0.0001)

Epoch 1/1000
625/625 [==============================] - 72s 112ms/step - loss: 0.6932 - binary_accuracy: 0.4907 - val_loss: 0.6932 - val_binary_accuracy: 0.5018
Epoch 2/1000
625/625 [==============================] - 20s 32ms/step - loss: 0.6930 - binary_accuracy: 0.5083 - val_loss: 0.6931 - val_binary_accuracy: 0.5024
Epoch 3/1000
625/625 [==============================] - 17s 27ms/step - loss: 0.6926 - binary_accuracy: 0.5418 - val_loss: 0.6930 - val_binary_accuracy: 0.5200
Epoch 4/1000
625/625 [==============================] - 13s 21ms/step - loss: 0.6911 - binary_accuracy: 0.5504 - val_loss: 0.6877 - val_binary_accuracy: 0.5298
Epoch 5/1000
625/625 [==============================] - 12s 20ms/step - loss: 0.6359 - binary_accuracy: 0.6591 - val_loss: 0.6521 - val_binary_accuracy: 0.6162
Epoch 6/1000
625/625 [==============================] - 12s 19ms/step - loss: 0.6591 - binary_accuracy: 0.5928 - val_loss: 0.6767 - val_binary_accuracy: 0.5489
Epoch 7/1000
625/625 [=================

KeyboardInterrupt: ignored

# [Text classification with an RNN](https://www.tensorflow.org/text/tutorials/text_classification_rnn)

In [ ]:
#same as above but different lr
compileAndFit(train_ds,val_ds,lambda:Sequential([
     layers.Embedding(vocablaryLength, 64, mask_zero=True),
     layers.Bidirectional( layers.LSTM(64)),
     layers.Dense(64, activation='relu'),
     layers.Dense(1)
]),0.0001)

Epoch 1/1000
625/625 [==============================] - 102s 138ms/step - loss: 0.5763 - binary_accuracy: 0.6854 - val_loss: 0.4164 - val_binary_accuracy: 0.8265
Epoch 2/1000
625/625 [==============================] - 38s 61ms/step - loss: 0.3242 - binary_accuracy: 0.8742 - val_loss: 0.3650 - val_binary_accuracy: 0.8588
Epoch 3/1000
625/625 [==============================] - 31s 49ms/step - loss: 0.2424 - binary_accuracy: 0.9090 - val_loss: 0.3166 - val_binary_accuracy: 0.8734
Epoch 4/1000
625/625 [==============================] - 28s 45ms/step - loss: 0.1907 - binary_accuracy: 0.9323 - val_loss: 0.3766 - val_binary_accuracy: 0.8438
Epoch 5/1000
625/625 [==============================] - 24s 39ms/step - loss: 0.1584 - binary_accuracy: 0.9452 - val_loss: 0.3343 - val_binary_accuracy: 0.8712
Epoch 6/1000
625/625 [==============================] - 25s 40ms/step - loss: 0.1291 - binary_accuracy: 0.9581 - val_loss: 0.3811 - val_binary_accuracy: 0.8682
Epoch 7/1000
625/625 [================

# [Text classification with an RNN 2](https://www.tensorflow.org/text/tutorials/text_classification_rnn)

In [ ]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
         layers.Embedding( vocablaryLength, 64, mask_zero=True),
         layers.Bidirectional(layers.LSTM(64,  return_sequences=True)),
         layers.Bidirectional(layers.LSTM(32)),
         layers.Dense(64, activation='relu'),
         layers.Dropout(0.5),
         layers.Dense(1)
        ]),0.0001)

Epoch 1/1000
625/625 [==============================] - 104s 139ms/step - loss: 0.5495 - binary_accuracy: 0.7029 - val_loss: 0.3438 - val_binary_accuracy: 0.8538
Epoch 2/1000
625/625 [==============================] - 48s 76ms/step - loss: 0.2881 - binary_accuracy: 0.8888 - val_loss: 0.3039 - val_binary_accuracy: 0.8704
Epoch 3/1000
625/625 [==============================] - 44s 71ms/step - loss: 0.2106 - binary_accuracy: 0.9255 - val_loss: 0.4117 - val_binary_accuracy: 0.8586
Epoch 4/1000
625/625 [==============================] - 43s 68ms/step - loss: 0.1668 - binary_accuracy: 0.9434 - val_loss: 0.3474 - val_binary_accuracy: 0.8726
Epoch 5/1000
625/625 [==============================] - 41s 66ms/step - loss: 0.1301 - binary_accuracy: 0.9586 - val_loss: 0.4318 - val_binary_accuracy: 0.8620
Epoch 6/1000
625/625 [==============================] - 41s 65ms/step - loss: 0.1039 - binary_accuracy: 0.9687 - val_loss: 0.4733 - val_binary_accuracy: 0.8654
Epoch 7/1000
625/625 [================

# [Bidirectional LSTM on IMDB](https://keras.io/examples/nlp/bidirectional_lstm_imdb/)

In [ ]:
compileAndFit( train_ds, val_ds,lambda:Sequential([
    layers.Embedding(vocablaryLength, 128),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1)
    ]),0.001)

Epoch 1/1000
625/625 [==============================] - 100s 143ms/step - loss: 0.5157 - binary_accuracy: 0.7441 - val_loss: 0.3968 - val_binary_accuracy: 0.8363
Epoch 2/1000
625/625 [==============================] - 45s 72ms/step - loss: 0.3348 - binary_accuracy: 0.8647 - val_loss: 0.3753 - val_binary_accuracy: 0.8454
Epoch 3/1000
625/625 [==============================] - 37s 60ms/step - loss: 0.2567 - binary_accuracy: 0.8992 - val_loss: 0.3622 - val_binary_accuracy: 0.8393
Epoch 4/1000
625/625 [==============================] - 33s 53ms/step - loss: 0.2030 - binary_accuracy: 0.9246 - val_loss: 0.3267 - val_binary_accuracy: 0.8640
Epoch 5/1000
625/625 [==============================] - 33s 52ms/step - loss: 0.1553 - binary_accuracy: 0.9450 - val_loss: 0.3848 - val_binary_accuracy: 0.8608
Epoch 6/1000
625/625 [==============================] - 32s 51ms/step - loss: 0.1526 - binary_accuracy: 0.9457 - val_loss: 0.3958 - val_binary_accuracy: 0.8650
Epoch 7/1000
625/625 [================

# [Getting Started with KerasNLP](https://keras.io/guides/keras_nlp/getting_started/)

In [ ]:
import GettingStartedwithKerasNLP

In [ ]:
bertModel="bert_base_en_uncased"

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(bertModel, sequence_length=sequence_length)

train_ds_bert =  raw_train_ds.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
val_ds_bert =   raw_val_ds.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

231508/231508 [==============================] - 0s 1us/step


In [ ]:
compileAndFit(train_ds_bert,val_ds_bert,lambda:GettingStartedwithKerasNLP.newModel(bertModel),5e-5)

Epoch 1/1000


625/625 [==============================] - 192s 252ms/step - loss: 0.2827 - binary_accuracy: 0.8806 - val_loss: 0.2393 - val_binary_accuracy: 0.9080
Epoch 2/1000
625/625 [==============================] - 105s 168ms/step - loss: 0.1388 - binary_accuracy: 0.9481 - val_loss: 0.2896 - val_binary_accuracy: 0.8926
Epoch 3/1000
625/625 [==============================] - 105s 168ms/step - loss: 0.0928 - binary_accuracy: 0.9665 - val_loss: 0.3556 - val_binary_accuracy: 0.9022
Epoch 4/1000
625/625 [==============================] - 105s 168ms/step - loss: 0.0542 - binary_accuracy: 0.9804 - val_loss: 0.4418 - val_binary_accuracy: 0.8950
Epoch 5/1000
625/625 [==============================] - 105s 169ms/step - loss: 0.0301 - binary_accuracy: 0.9896 - val_loss: 0.4317 - val_binary_accuracy: 0.9010
Epoch 6/1000
625/625 [==============================] - 35s 55ms/step - loss: 0.1176 - binary_accuracy: 0.9607
Epoch 1/1000


625/625 [==============================] - 194s 252ms/step - loss: 0.2855 - binary_accuracy: 0.8763 - val_loss: 0.2650 - val_binary_accuracy: 0.9010
Epoch 2/1000
625/625 [==============================] - 106s 169ms/step - loss: 0.1396 - binary_accuracy: 0.9483 - val_loss: 0.2908 - val_binary_accuracy: 0.8916
Epoch 3/1000
625/625 [==============================] - 106s 169ms/step - loss: 0.0871 - binary_accuracy: 0.9693 - val_loss: 0.5411 - val_binary_accuracy: 0.8624
Epoch 4/1000
625/625 [==============================] - 106s 169ms/step - loss: 0.0587 - binary_accuracy: 0.9782 - val_loss: 0.4004 - val_binary_accuracy: 0.9032
Epoch 5/1000
625/625 [==============================] - 106s 170ms/step - loss: 0.0353 - binary_accuracy: 0.9879 - val_loss: 0.4029 - val_binary_accuracy: 0.9054
Epoch 6/1000
625/625 [==============================] - 106s 169ms/step - loss: 0.0343 - binary_accuracy: 0.9880 - val_loss: 0.4245 - val_binary_accuracy: 0.9092
Epoch 7/1000
625/625 [===================

625/625 [==============================] - 200s 255ms/step - loss: 0.2819 - binary_accuracy: 0.8810 - val_loss: 0.2275 - val_binary_accuracy: 0.9104
Epoch 2/1000
625/625 [==============================] - 108s 173ms/step - loss: 0.1446 - binary_accuracy: 0.9455 - val_loss: 0.2705 - val_binary_accuracy: 0.8994
Epoch 3/1000
625/625 [==============================] - 108s 173ms/step - loss: 0.0837 - binary_accuracy: 0.9705 - val_loss: 0.4245 - val_binary_accuracy: 0.8824
Epoch 4/1000
625/625 [==============================] - 108s 173ms/step - loss: 0.0560 - binary_accuracy: 0.9794 - val_loss: 0.3422 - val_binary_accuracy: 0.9062
Epoch 5/1000
625/625 [==============================] - 108s 172ms/step - loss: 0.0361 - binary_accuracy: 0.9872 - val_loss: 0.4297 - val_binary_accuracy: 0.8996
Epoch 6/1000
625/625 [==============================] - 35s 56ms/step - loss: 0.1090 - binary_accuracy: 0.9663
Time,s	Training accuracy	Validation accuracy	Epochs
719	0.961	0.908	6
1781	0.998	0.91	16
740

**Section - Build and train your own transformer from scratch**

In [25]:
def newTransformerFromScratch(vocablaryLength,sequence_length):
    input = keras.Input(shape=(None,),dtype="int32",name="token_ids")
    outputs = keras_nlp.layers.TokenAndPositionEmbedding(
        vocabulary_size=vocablaryLength,    sequence_length= sequence_length,    embedding_dim=64)(input)
    outputs = keras_nlp.layers.TransformerEncoder(num_heads=2,intermediate_dim=128,dropout=0.1)(outputs)
    # Use "[START]" token to classify
    x =  layers.Dropout(0.1)(outputs[:, 0, :])
    outputs =  layers.Dense(1)(x)
    return keras.Model(inputs=input,outputs=outputs)

In [26]:
compileAndFit(train_ds ,val_ds ,lambda:newTransformerFromScratch(vocablaryLength,sequence_length),5e-5)

Epoch 1/1000
625/625 [==============================] - 78s 110ms/step - loss: 0.7151 - binary_accuracy: 0.5131 - val_loss: 0.6834 - val_binary_accuracy: 0.5789
Epoch 2/1000
625/625 [==============================] - 24s 38ms/step - loss: 0.6685 - binary_accuracy: 0.5865 - val_loss: 0.6175 - val_binary_accuracy: 0.6480
Epoch 3/1000
625/625 [==============================] - 14s 23ms/step - loss: 0.4552 - binary_accuracy: 0.7898 - val_loss: 0.3750 - val_binary_accuracy: 0.8363
Epoch 4/1000
625/625 [==============================] - 13s 21ms/step - loss: 0.3055 - binary_accuracy: 0.8702 - val_loss: 0.3788 - val_binary_accuracy: 0.8353
Epoch 5/1000
625/625 [==============================] - 10s 16ms/step - loss: 0.2570 - binary_accuracy: 0.8952 - val_loss: 0.3381 - val_binary_accuracy: 0.8590
Epoch 6/1000
625/625 [==============================] - 10s 17ms/step - loss: 0.2262 - binary_accuracy: 0.9105 - val_loss: 0.3436 - val_binary_accuracy: 0.8626
Epoch 7/1000
625/625 [=================

# [Text Classification using FNet](https://keras.io/examples/nlp/fnet_classification_with_keras_nlp/)

In [18]:
import TextClassificationusingFNet

In [19]:
compileAndFit(train_ds,val_ds,lambda:TextClassificationusingFNet.newFnet(max_features,sequence_length),0.001)

Epoch 1/1000


625/625 [==============================] - 87s 123ms/step - loss: 0.5382 - binary_accuracy: 0.6959 - val_loss: 0.3735 - val_binary_accuracy: 0.8355
Epoch 2/1000
625/625 [==============================] - 34s 54ms/step - loss: 0.3033 - binary_accuracy: 0.8716 - val_loss: 0.3993 - val_binary_accuracy: 0.8411
Epoch 3/1000
625/625 [==============================] - 25s 39ms/step - loss: 0.1985 - binary_accuracy: 0.9219 - val_loss: 0.4089 - val_binary_accuracy: 0.8554
Epoch 4/1000
625/625 [==============================] - 19s 30ms/step - loss: 0.1247 - binary_accuracy: 0.9520 - val_loss: 0.5024 - val_binary_accuracy: 0.8383
Epoch 5/1000
625/625 [==============================] - 17s 27ms/step - loss: 0.0878 - binary_accuracy: 0.9693 - val_loss: 0.5258 - val_binary_accuracy: 0.8460
Epoch 6/1000
625/625 [==============================] - 16s 25ms/step - loss: 0.0654 - binary_accuracy: 0.9764 - val_loss: 0.5530 - val_binary_accuracy: 0.8488
Epoch 7/1000
625/625 [==============================

625/625 [==============================] - 65s 94ms/step - loss: 0.5387 - binary_accuracy: 0.7012 - val_loss: 0.3675 - val_binary_accuracy: 0.8429
Epoch 2/1000
625/625 [==============================] - 25s 39ms/step - loss: 0.2970 - binary_accuracy: 0.8742 - val_loss: 0.3424 - val_binary_accuracy: 0.8576
Epoch 3/1000
625/625 [==============================] - 18s 29ms/step - loss: 0.1913 - binary_accuracy: 0.9241 - val_loss: 0.3754 - val_binary_accuracy: 0.8630
Epoch 4/1000
625/625 [==============================] - 19s 31ms/step - loss: 0.1241 - binary_accuracy: 0.9524 - val_loss: 0.4709 - val_binary_accuracy: 0.8524
Epoch 5/1000
625/625 [==============================] - 16s 25ms/step - loss: 0.0813 - binary_accuracy: 0.9701 - val_loss: 0.6437 - val_binary_accuracy: 0.8255
Epoch 6/1000
625/625 [==============================] - 17s 27ms/step - loss: 0.0609 - binary_accuracy: 0.9787 - val_loss: 0.5961 - val_binary_accuracy: 0.8448
Epoch 7/1000
625/625 [==============================]

625/625 [==============================] - 63s 90ms/step - loss: 0.5796 - binary_accuracy: 0.6565 - val_loss: 0.4191 - val_binary_accuracy: 0.8105
Epoch 2/1000
625/625 [==============================] - 25s 40ms/step - loss: 0.3093 - binary_accuracy: 0.8721 - val_loss: 0.3307 - val_binary_accuracy: 0.8676
Epoch 3/1000
625/625 [==============================] - 18s 28ms/step - loss: 0.2036 - binary_accuracy: 0.9223 - val_loss: 0.3897 - val_binary_accuracy: 0.8568
Epoch 4/1000
625/625 [==============================] - 19s 30ms/step - loss: 0.1426 - binary_accuracy: 0.9463 - val_loss: 0.3949 - val_binary_accuracy: 0.8682
Epoch 5/1000
625/625 [==============================] - 19s 30ms/step - loss: 0.0894 - binary_accuracy: 0.9663 - val_loss: 0.4883 - val_binary_accuracy: 0.8502
Epoch 6/1000
625/625 [==============================] - 16s 25ms/step - loss: 0.0624 - binary_accuracy: 0.9783 - val_loss: 0.6359 - val_binary_accuracy: 0.8273
Epoch 7/1000
625/625 [==============================]

In [20]:
compileAndFit(train_ds,val_ds,lambda:TextClassificationusingFNet.newTransformer(max_features,sequence_length),0.001)

Epoch 1/1000
625/625 [==============================] - 73s 100ms/step - loss: 0.4342 - binary_accuracy: 0.7851 - val_loss: 0.3053 - val_binary_accuracy: 0.8754
Epoch 2/1000
625/625 [==============================] - 28s 44ms/step - loss: 0.2328 - binary_accuracy: 0.9078 - val_loss: 0.3075 - val_binary_accuracy: 0.8716
Epoch 3/1000
625/625 [==============================] - 22s 35ms/step - loss: 0.1676 - binary_accuracy: 0.9384 - val_loss: 0.5366 - val_binary_accuracy: 0.8125
Epoch 4/1000
625/625 [==============================] - 21s 33ms/step - loss: 0.3183 - binary_accuracy: 0.8328 - val_loss: 0.6098 - val_binary_accuracy: 0.6581
Epoch 5/1000
625/625 [==============================] - 17s 28ms/step - loss: 0.5186 - binary_accuracy: 0.7627 - val_loss: 0.5055 - val_binary_accuracy: 0.7598
Epoch 6/1000
625/625 [==============================] - 8s 13ms/step - loss: 0.2270 - binary_accuracy: 0.9301
Epoch 1/1000
625/625 [==============================] - 74s 101ms/step - loss: 0.4203 - b

# [Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/)

In [ ]:
import TextclassificationwithTransformer

In [ ]:
compileAndFit(train_ds ,val_ds ,lambda:TextclassificationwithTransformer.newTransformer(sequence_length,max_features),0.001)

Epoch 1/1000
625/625 [==============================] - 78s 115ms/step - loss: 0.4197 - binary_accuracy: 0.7928 - val_loss: 0.2901 - val_binary_accuracy: 0.8800
Epoch 2/1000
625/625 [==============================] - 27s 44ms/step - loss: 0.2289 - binary_accuracy: 0.9107 - val_loss: 0.3045 - val_binary_accuracy: 0.8746
Epoch 3/1000
625/625 [==============================] - 19s 31ms/step - loss: 0.1661 - binary_accuracy: 0.9403 - val_loss: 0.3752 - val_binary_accuracy: 0.8572
Epoch 4/1000
625/625 [==============================] - 15s 24ms/step - loss: 0.1208 - binary_accuracy: 0.9586 - val_loss: 0.4431 - val_binary_accuracy: 0.8588
Epoch 5/1000
625/625 [==============================] - 15s 24ms/step - loss: 0.0890 - binary_accuracy: 0.9724 - val_loss: 0.5866 - val_binary_accuracy: 0.8456
Epoch 6/1000
625/625 [==============================] - 4s 7ms/step - loss: 0.2173 - binary_accuracy: 0.9219
Epoch 1/1000
625/625 [==============================] - 63s 92ms/step - loss: 0.4533 - bin

# [Review Classification using Active Learning](https://keras.io/examples/nlp/active_learning_review_classification/)

In [ ]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
            layers.Embedding(vocablaryLength, output_dim=128),
            layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
            layers.GlobalMaxPool1D(),
            layers.Dense(20, activation="relu"),
            layers.Dropout(0.5),
            layers.Dense(1 )
  ]),0.001)

Epoch 1/1000
625/625 [==============================] - 82s 121ms/step - loss: 0.4403 - binary_accuracy: 0.7962 - val_loss: 0.3150 - val_binary_accuracy: 0.8694
Epoch 2/1000
625/625 [==============================] - 33s 52ms/step - loss: 0.2485 - binary_accuracy: 0.9115 - val_loss: 0.3279 - val_binary_accuracy: 0.8602
Epoch 3/1000
625/625 [==============================] - 25s 40ms/step - loss: 0.1728 - binary_accuracy: 0.9438 - val_loss: 0.3859 - val_binary_accuracy: 0.8626
Epoch 4/1000
625/625 [==============================] - 22s 35ms/step - loss: 0.1199 - binary_accuracy: 0.9628 - val_loss: 0.4513 - val_binary_accuracy: 0.8522
Epoch 5/1000
625/625 [==============================] - 19s 31ms/step - loss: 0.0898 - binary_accuracy: 0.9731 - val_loss: 0.5152 - val_binary_accuracy: 0.8546
Epoch 6/1000
625/625 [==============================] - 10s 15ms/step - loss: 0.2436 - binary_accuracy: 0.9243
Epoch 1/1000
625/625 [==============================] - 67s 99ms/step - loss: 0.4615 - b

# [Load text](https://www.tensorflow.org/tutorials/load_data/text)

In [ ]:
compileAndFit(train_ds,val_ds,lambda:Sequential([
      layers.Embedding(vocablaryLength, 64),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(1)
  ]),0.001)

Epoch 1/1000
625/625 [==============================] - 57s 82ms/step - loss: 0.4913 - binary_accuracy: 0.7508 - val_loss: 0.3605 - val_binary_accuracy: 0.8407
Epoch 2/1000
625/625 [==============================] - 14s 23ms/step - loss: 0.2627 - binary_accuracy: 0.8915 - val_loss: 0.3088 - val_binary_accuracy: 0.8722
Epoch 3/1000
625/625 [==============================] - 10s 16ms/step - loss: 0.1442 - binary_accuracy: 0.9499 - val_loss: 0.3310 - val_binary_accuracy: 0.8710
Epoch 4/1000
625/625 [==============================] - 9s 14ms/step - loss: 0.0591 - binary_accuracy: 0.9858 - val_loss: 0.3801 - val_binary_accuracy: 0.8678
Epoch 5/1000
625/625 [==============================] - 7s 11ms/step - loss: 0.0180 - binary_accuracy: 0.9983 - val_loss: 0.4330 - val_binary_accuracy: 0.8678
Epoch 6/1000
625/625 [==============================] - 9s 14ms/step - loss: 0.0052 - binary_accuracy: 0.9998 - val_loss: 0.4788 - val_binary_accuracy: 0.8666
Epoch 7/1000
625/625 [=====================

# [Text classification from scratch](https://keras.io/examples/nlp/text_classification_from_scratch/)

In [ ]:
compileAndFit(train_ds,val_ds,lambda:Sequential(
    [
        layers.Embedding(vocablaryLength, 128),
        layers.Dropout(0.5),
        layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3),
        layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3),
        layers.GlobalMaxPooling1D(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(1, name="predictions"),
    ]),0.001)

Epoch 1/1000
625/625 [==============================] - 58s 82ms/step - loss: 0.5136 - binary_accuracy: 0.7039 - val_loss: 0.3277 - val_binary_accuracy: 0.8544
Epoch 2/1000
625/625 [==============================] - 20s 32ms/step - loss: 0.2567 - binary_accuracy: 0.8961 - val_loss: 0.3355 - val_binary_accuracy: 0.8638
Epoch 3/1000
625/625 [==============================] - 12s 19ms/step - loss: 0.1542 - binary_accuracy: 0.9420 - val_loss: 0.4502 - val_binary_accuracy: 0.8560
Epoch 4/1000
625/625 [==============================] - 11s 18ms/step - loss: 0.0846 - binary_accuracy: 0.9693 - val_loss: 0.5606 - val_binary_accuracy: 0.8570
Epoch 5/1000
625/625 [==============================] - 8s 12ms/step - loss: 0.0558 - binary_accuracy: 0.9806 - val_loss: 0.6114 - val_binary_accuracy: 0.8518
Epoch 6/1000
625/625 [==============================] - 8s 13ms/step - loss: 0.0394 - binary_accuracy: 0.9862 - val_loss: 0.7370 - val_binary_accuracy: 0.8550
Epoch 7/1000
625/625 [====================

# [Text classification with Switch Transformer](https://keras.io/examples/nlp/text_classification_with_switch_transformer/)

In [ ]:
import TextclassificationwithSwitchTransformer


In [ ]:
tmp=TextclassificationwithSwitchTransformer.newClassifier(batch_size,sequence_length,max_features)

In [ ]:
tmp.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [ ]:
#https://keras.io/examples/nlp/text_classification_with_switch_transformer/
compileAndFit(train_ds,val_ds,lambda:TextclassificationwithSwitchTransformer.newClassifier(batch_size,sequence_length,max_features),0.001)


Epoch 1/1000
625/625 [==============================] - 87s 120ms/step - loss: 1.4191 - binary_accuracy: 0.8015 - val_loss: 1.2911 - val_binary_accuracy: 0.8786
Epoch 2/1000
625/625 [==============================] - 30s 47ms/step - loss: 1.2434 - binary_accuracy: 0.9053 - val_loss: 1.2954 - val_binary_accuracy: 0.8752
Epoch 3/1000
625/625 [==============================] - 24s 38ms/step - loss: 1.1937 - binary_accuracy: 0.9279 - val_loss: 1.3336 - val_binary_accuracy: 0.8742
Epoch 4/1000
625/625 [==============================] - 21s 33ms/step - loss: 1.1352 - binary_accuracy: 0.9528 - val_loss: 1.4442 - val_binary_accuracy: 0.8602
Epoch 5/1000
625/625 [==============================] - 19s 29ms/step - loss: 1.1044 - binary_accuracy: 0.9664 - val_loss: 1.5095 - val_binary_accuracy: 0.8552
Epoch 6/1000
625/625 [==============================] - 8s 13ms/step - loss: 1.2146 - binary_accuracy: 0.9276
Epoch 1/1000
625/625 [==============================] - 72s 98ms/step - loss: 1.4454 - bi

# [End-to-end Masked Language Modeling with BERT](https://keras.io/examples/nlp/masked_language_modeling/)

In [ ]:
import EndtoendMaskedLanguageModelingwithBERT

In [ ]:
compileAndFit(train_ds,val_ds,lambda:EndtoendMaskedLanguageModelingwithBERT.newBert(sequence_length,vocablaryLength),0.001)

Epoch 1/1000
625/625 [==============================] - 76s 113ms/step - loss: 0.7019 - binary_accuracy: 0.5044 - val_loss: 0.6929 - val_binary_accuracy: 0.5074
Epoch 2/1000
625/625 [==============================] - 25s 40ms/step - loss: 0.6955 - binary_accuracy: 0.5066 - val_loss: 0.7222 - val_binary_accuracy: 0.4924
Epoch 3/1000
625/625 [==============================] - 18s 28ms/step - loss: 0.6162 - binary_accuracy: 0.6240 - val_loss: 0.4274 - val_binary_accuracy: 0.8172
Epoch 4/1000
625/625 [==============================] - 14s 23ms/step - loss: 0.3316 - binary_accuracy: 0.8623 - val_loss: 0.3163 - val_binary_accuracy: 0.8720
Epoch 5/1000
625/625 [==============================] - 15s 24ms/step - loss: 0.1986 - binary_accuracy: 0.9262 - val_loss: 0.3893 - val_binary_accuracy: 0.8578
Epoch 6/1000
625/625 [==============================] - 12s 20ms/step - loss: 0.1256 - binary_accuracy: 0.9560 - val_loss: 0.4669 - val_binary_accuracy: 0.8580
Epoch 7/1000
625/625 [=================

# [Text classification with TensorFlow Hub: Movie reviews](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub)
# [Text Classification with Movie Reviews](https://www.tensorflow.org/hub/tutorials/tf2_text_classification)

In [ ]:
compileAndFit(raw_train_ds,raw_val_ds,lambda:Sequential([
         hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2", input_shape=[],dtype=tf.string, trainable=True),
         layers.Dense(16, activation='relu'),
         layers.Dense(1)
        ]),0.001)

Epoch 1/1000
625/625 [==============================] - 116s 175ms/step - loss: 0.3719 - binary_accuracy: 0.8360 - val_loss: 0.2625 - val_binary_accuracy: 0.8914
Epoch 2/1000
625/625 [==============================] - 58s 92ms/step - loss: 0.1596 - binary_accuracy: 0.9412 - val_loss: 0.2713 - val_binary_accuracy: 0.8934
Epoch 3/1000
625/625 [==============================] - 38s 62ms/step - loss: 0.0806 - binary_accuracy: 0.9764 - val_loss: 0.3133 - val_binary_accuracy: 0.8866
Epoch 4/1000
625/625 [==============================] - 26s 41ms/step - loss: 0.0340 - binary_accuracy: 0.9920 - val_loss: 0.3778 - val_binary_accuracy: 0.8820
Epoch 5/1000
625/625 [==============================] - 19s 31ms/step - loss: 0.0118 - binary_accuracy: 0.9984 - val_loss: 0.4397 - val_binary_accuracy: 0.8820
Epoch 6/1000
625/625 [==============================] - 19s 30ms/step - loss: 0.0038 - binary_accuracy: 0.9998 - val_loss: 0.4928 - val_binary_accuracy: 0.8826
Epoch 7/1000
625/625 [================

Just to test if the dense layer makes sense, it does not in the averaging model

In [17]:
compileAndFit(raw_train_ds,raw_val_ds,lambda:Sequential([
         hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2", input_shape=[],dtype=tf.string, trainable=True),
         layers.Dense(1)
        ]),0.001)

Epoch 1/1000
625/625 [==============================] - 120s 182ms/step - loss: 0.4178 - binary_accuracy: 0.8361 - val_loss: 0.2894 - val_binary_accuracy: 0.8872
Epoch 2/1000
625/625 [==============================] - 58s 93ms/step - loss: 0.2057 - binary_accuracy: 0.9254 - val_loss: 0.2610 - val_binary_accuracy: 0.8952
Epoch 3/1000
625/625 [==============================] - 38s 61ms/step - loss: 0.1310 - binary_accuracy: 0.9585 - val_loss: 0.2542 - val_binary_accuracy: 0.8952
Epoch 4/1000
625/625 [==============================] - 26s 41ms/step - loss: 0.0819 - binary_accuracy: 0.9790 - val_loss: 0.2888 - val_binary_accuracy: 0.8884
Epoch 5/1000
625/625 [==============================] - 20s 32ms/step - loss: 0.0507 - binary_accuracy: 0.9893 - val_loss: 0.2923 - val_binary_accuracy: 0.8934
Epoch 6/1000
625/625 [==============================] - 19s 30ms/step - loss: 0.0300 - binary_accuracy: 0.9954 - val_loss: 0.3211 - val_binary_accuracy: 0.8880
Epoch 7/1000
625/625 [================

# [Classify text with BERT](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)

In [33]:
tfhub_handle_encoder = 'https://tfhub.dev/google/electra_base/2'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [34]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)


In [38]:
compileAndFit(raw_train_ds,raw_val_ds,build_classifier_model,0.00003)

Epoch 1/1000
625/625 [==============================] - 347s 467ms/step - loss: 0.2844 - binary_accuracy: 0.8784 - val_loss: 0.2583 - val_binary_accuracy: 0.8920
Epoch 2/1000
625/625 [==============================] - 276s 442ms/step - loss: 0.1658 - binary_accuracy: 0.9376 - val_loss: 0.2681 - val_binary_accuracy: 0.9091
Epoch 3/1000
625/625 [==============================] - 272s 436ms/step - loss: 0.0933 - binary_accuracy: 0.9683 - val_loss: 0.3673 - val_binary_accuracy: 0.9054
Epoch 4/1000
625/625 [==============================] - 271s 433ms/step - loss: 0.0589 - binary_accuracy: 0.9796 - val_loss: 0.4661 - val_binary_accuracy: 0.9014
Epoch 5/1000
625/625 [==============================] - 253s 405ms/step - loss: 0.0381 - binary_accuracy: 0.9876 - val_loss: 0.4658 - val_binary_accuracy: 0.9087
Epoch 6/1000
625/625 [==============================] - 251s 401ms/step - loss: 0.0272 - binary_accuracy: 0.9913 - val_loss: 0.5224 - val_binary_accuracy: 0.9058
Epoch 7/1000
625/625 [======